<a href="https://colab.research.google.com/github/solanods/pandas/blob/master/ponto_segunda_quinzena.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:
# Imports
from google.colab import drive
drive.mount('/content/drive')

from re import search

import pandas as pd
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [93]:
# Carregamento dos dados
df = pd.read_excel('/content/drive/MyDrive/PRMS/PRMS_01_JAN_2022.xlsm', sheet_name='RMOP')

/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [94]:
# Exibe a planilha com todos RMOPS
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,ABREU,NaN,NaN,NaN,NaN,NaN,NaN,RE,957,JANEIRO,NaN,2022,Telefone:,991 015 602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13234,B27,CovUtil12_2325,FOL,27,2022-01-27 00:00:00,FOL,FOL,_,FOL,NaN,,,FIXO,,,,NaN,NaN,NaN,NaN,NaN,_,0.0,0.0,NaN,NaN,NaN,FOL,FOL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HEFT,HE folga TARDE,hora extra vespertina em dia de folga
13235,B28,CovUtil12_2325.2,FOL,28,2022-01-28 00:00:00,FOL,FOL,_,FOL,NaN,,,FIXO,,,,NaN,NaN,NaN,NaN,NaN,_,0.0,0.0,NaN,NaN,NaN,FOL,FOL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [95]:
# Selecionar colunas de interesse
df = df.iloc[:,[3,5,6,7,8,11,16,17,18,19]]

# Renomear colunas
df = df.rename({'Unnamed: 3': 'DIA',
                'Unnamed: 5': 'PREVISTO',
                'Unnamed: 6': 'ESCALA',
                'Unnamed: 7': 'TOTVS',
                'Unnamed: 8': 'JORNADA',
                'Unnamed: 11': 'ENTRADA',
                'Unnamed: 16': 'CODIGO',
                'Unnamed: 17': 'DESCRICAO',
                'Unnamed: 18': 'OBSERVACOES',
                'Unnamed: 19': 'RE_SGT'}, axis = 1)

# Remover nan
df = df.fillna('-')

In [62]:
# Mascaras da ESCALA
revezamento = df['ESCALA'].str.contains('^[A-Z]{1}\d{1}$')==True
semanal = df['ESCALA'].str.contains('^[A-Z]{1}\d{2}$')==True
quatro_dois_quatro = df['ESCALA'].str.contains('^[A-Z]{2}$')==True
aeromovel = df['ESCALA'].str.contains('^[A-Z]{3}$')==True
era_fg = df['ESCALA'].str.contains('^[A-Z]{2}\d{1}$')==True

# Mascaras da JORNADA
manha = df['JORNADA'].str.endswith('M')==True
tarde = df['JORNADA'].str.endswith('T')==True
noite = df['JORNADA'].str.endswith('N')==True
folga = df['JORNADA'].str.endswith('FOL')==True

# Mascaras do CODIGO
permuta = df['CODIGO'].str.contains('C500')== True
troca = df['CODIGO'].str.contains('C129')== True


# TROCA_PERMUTA 
df['TROCA_PERMUTA'] = ''

# Permutas
df.loc[manha & permuta, 'TROCA_PERMUTA'] = '01850'
df.loc[tarde & permuta, 'TROCA_PERMUTA'] = '01851'
df.loc[noite & permuta, 'TROCA_PERMUTA'] = '01956'
df.loc[semanal & manha & permuta, 'TROCA_PERMUTA'] = '01613'
df.loc[semanal & tarde & permuta, 'TROCA_PERMUTA'] = '01614'

# Trocas
df.loc[manha & troca, 'TROCA_PERMUTA'] = '01840'
df.loc[tarde & troca, 'TROCA_PERMUTA'] = '01841'
df.loc[noite & troca, 'TROCA_PERMUTA'] = '01845'
df.loc[folga & troca, 'TROCA_PERMUTA'] = '01768'
df.loc[semanal & manha & troca, 'TROCA_PERMUTA'] = '01610'
df.loc[semanal & tarde & troca, 'TROCA_PERMUTA'] = '01611'
df.loc[semanal & folga & troca, 'TROCA_PERMUTA'] = '01612'

# ESCALA_TOTVS
df['ESCALA_TOTVS'] = ''

# Mascaras da ESCALA
revezamento_manha = df['ESCALA'].str.contains('^[A-I]{1}\d{1}$')==True
revezamento_tarde = df['ESCALA'].str.contains('^[J-R]{1}\d{1}$')==True
semanal_manha = df['ESCALA'].str.startswith('X')==True
semanal_tarde = df['ESCALA'].str.startswith('Y')==True
quatro_dois_quatro_manha = df['ESCALA'].str.contains('^[A-I][A-B]$')==True
quatro_dois_quatro_tarde = df['ESCALA'].str.contains('^[J-R][A-B]$')==True
aeromovel_manha = df['ESCALA'].str.contains('^[A-E][A-Z]{2}$')==True
aeromovel_tarde = df['ESCALA'].str.contains('^[J-N][A-Z]{2}$')==True


df.loc[revezamento_manha, 'ESCALA_TOTVS'] = '01116'
df.loc[revezamento_tarde, 'ESCALA_TOTVS'] = '01729'
df.loc[semanal_manha, 'ESCALA_TOTVS'] = '01600'
df.loc[semanal_tarde, 'ESCALA_TOTVS'] = '01601'
df.loc[quatro_dois_quatro_manha, 'ESCALA_TOTVS'] = '01622'
df.loc[quatro_dois_quatro_tarde, 'ESCALA_TOTVS'] = '01630'
df.loc[aeromovel_manha, 'ESCALA_TOTVS'] = '01799'
df.loc[aeromovel_tarde, 'ESCALA_TOTVS'] = '01811'

In [96]:
  # Obter lista com nome de todos operadores
  operadores = []
  for linha in df.index:
    if search ('[A-Z]+', str(df['DIA'][linha])):
      operadores.append(df['DIA'][linha])

  operadores = [operador for operador in operadores if operador.isupper()]

In [97]:
# Extrair rmop
i = 3
j =38
R = []
df['NOME'] = ''
df['RE'] = ''
for operador in operadores:
  df['NOME'].loc[i:j] = operador
  rmop_operador = df.loc[i:j]
  R.append(rmop_operador)
  df['RE'].loc[i:j] = df['ENTRADA'].loc[i:j].iloc[0]
  i+=66
  j+=66

In [98]:
# Primeira quinzena
R[112].iloc[:20,:]

,DIA,PREVISTO,ESCALA,TOTVS,JORNADA,ENTRADA,CODIGO,DESCRICAO,OBSERVACOES,RE_SGT,NOME,RE
7395,MADALENA,-,-,-,-,972,997 519 683,-,-,-,MADALENA,972
7396,-,-,-,-,-,-,-,-,-,-,MADALENA,972
7397,Folgas:,Prev,-,4,Real,-,-,-,-,-,MADALENA,972
7398,Dia,Escala,-,-,-,-,Código,-,Observações,Re,MADALENA,972
7399,-,PREV,-,TOTVS,Jor,Entrada,Nº,Descrição,-,Resp,MADALENA,972
7400,1,FER,FER,_,FER,,-,-,-,-,MADALENA,972
7401,2,FER,FER,_,FER,,-,-,-,-,MADALENA,972
7402,3,FER,FER,_,FER,,-,-,-,-,MADALENA,972
7403,4,FER,FER,_,FER,,-,-,-,-,MADALENA,972
7404,5,FER,FER,_,FER,,-,-,-,-,MADALENA,972


In [68]:
# Segunda quinzena
R[112].iloc[20:,:]

,DIA,PREVISTO,ESCALA,TOTVS,JORNADA,ENTRADA,CODIGO,DESCRICAO,OBSERVACOES,RE_SGT,TROCA_PERMUTA,ESCALA_TOTVS,NOME,RE
7415,16,FER,FER,_,FER,,-,-,-,-,,,MADALENA,972
7416,17,FER,FER,_,FER,,-,-,-,-,,,MADALENA,972
7417,18,TRAB,ASS,_,ASS,,-,-,-,-,,01799,MADALENA,972
7418,19,TRAB,ASS,_,ASS,,-,-,-,-,,01799,MADALENA,972
7419,20,TRAB,ASS,_,ASS,,-,-,-,-,,01799,MADALENA,972
7420,21,TRAB,Y10,_,07T,13:15:00,C083,Troca Missão,Ajuste de efetivo,2923,,01601,MADALENA,972
7421,22,FOL,Y10,_,07T,12:40:00,HEFT,HE folga TARDE,Necessidade Operacional,2923,,01601,MADALENA,972
7422,23,FOL,Y10,_,FOL,,-,-,-,-,,01601,MADALENA,972
7423,24,TRAB,Y10,_,26T,12:10:00,-,-,-,-,,01601,MADALENA,972
7424,25,TRAB,Y10,_,26T,12:10:00,C083,Troca Missão,c/Freitas,2923,,01601,MADALENA,972


In [81]:
# Buscar valores deslocados

troca_desloc = df['OBSERVACOES'].str.contains('C129')==True
permuta_desloc = df['OBSERVACOES'].str.contains('C500')==True


In [82]:
df[troca_desloc]

,DIA,PREVISTO,ESCALA,TOTVS,JORNADA,ENTRADA,CODIGO,DESCRICAO,OBSERVACOES,RE_SGT,TROCA_PERMUTA,ESCALA_TOTVS,NOME,RE
695,28,FOL,Q1,17-Q,FAT,COLOCAR,C209,Atest Acomp,IRMÃ - C129/Por dia 30,2681/2665,,01729,ANA CAROLINA,2922
1206,11,FOL,JA,9-I,TNH,13:15:00,C083,Troca Missão,CARP - B + C129 por dia 17,2681/561,,01630,ARTUR,2934
1207,12,FOL,JA,9-I,04T,12:00:00,C083,Troca Missão,CARP - B + C129 por dia 18,2681,,01630,ARTUR,2934
1208,13,FOL,JA,9-I,INST1,TRAR TR,C083,Troca Missão,CARP - B + C129 por dia 19,2681/2445,,01630,ARTUR,2934
1409,16,FOL,MA,3-C,NPT,21:30:00,C129,Troca Jorn,C129 por dia 10,2681,01841,01630,BARCELLOS,542
5958,11,FOL,R4,15-O,FAT,COLOCAR,C208,Atestado,(C129 Por dia 15),252,,01729,JUBIRA,1481
5960,13,FOL,R4,15-O,FAT,COLOCAR,C208,Atestado,(C129 Por dia 16),252,,01729,JUBIRA,1481
6042,29,TRAB,CBB,8-H,AI1,10:00:00,HEFM,HE folga MANHÃ,C129 /Por dia 22 - NEC. OPERAC.,2681/2665,,01799,JULIANA,2833
7149,14,FOL,E3,3-C,26M,05:10:00,C129,Troca Jorn,C129 Por dia 22 (C300 - Autorizada entrada às ...,2681,01840,01116,LINO,1329
8062,3,FOL,OA,9-I,INST1,08:00:00,TRN,Lançar em jornada,Módulo IX - Preparação de Trens + C129 por dia 9,2681,,01630,MATHEUS,2415


In [76]:
df[permuta_desloc]

,DIA,PREVISTO,ESCALA,TOTVS,JORNADA,ENTRADA,CODIGO,DESCRICAO,OBSERVACOES,RE_SGT,TROCA_PERMUTA,ESCALA_TOTVS,NOME,RE
147,8,TRAB,D2,5-E,18N,21:30:00,TRN,Lançar em jornada,Novos Operadores + C500,2681,,01116,AG-HUR,538
1935,14,FOL,CAA,8-H,AT1,14:00:00,C129,Troca Jorn,Por dia 17 + C500 com Jonas,2681/2398,,01799,CARMEN,543
2400,17,FOL,P5,1-A,FAM,COLOCAR,C129,Troca Jorn,Por dia 1 + C500 Com Silvana+C208 COVID,2681/2445/2906,01840,01729,CLAUDETE MOURA,1434
3320,13,TRAB,G5,17-Q,09N,22:20:00,C500,Permuta,C083 com Luís + C500 com Roger,2735,01956,01116,DORNELLES,2453
4246,15,FOL,KAA,5-E,AT1,14:00:00,C129,Troca Jorn,Por dia 22 + C500 com Jonas,2681/2398,,01811,FRANCISCO,554
4247,16,FOL,KAA,5-E,AT2,14:00:00,C129,Troca Jorn,Por dia 23 + C500 com Jonas,2681/2398,,01811,FRANCISCO,554
5562,11,TRAB,E5,3-C,FAN,COLOCAR,C208,Atestado,C500 Com Venhofen+Suspeita COVID,2681/2945,,01116,JAQUELINE,2676
5563,12,TRAB,E5,3-C,FAN,COLOCAR,C208,Atestado,C500+Suspeita COVID,2681/2945,,01116,JAQUELINE,2676
5699,16,TRAB,JAA,9-I,AM1,06:00:00,C083,Troca Missão,e C500 com Francisco,2681/2398,,01811,JONAS,2882
5700,17,TRAB,JAA,9-I,26N,21:30:00,TTV,Teste Trem Via,Aeromóvel + C500,542,,01811,JONAS,2882


In [77]:
#Criar a funcao de busca RMOP por RE

def RE(number):
  return df.query(f'RE == {number}')
  

In [85]:
RE(1434)

,DIA,PREVISTO,ESCALA,TOTVS,JORNADA,ENTRADA,CODIGO,DESCRICAO,OBSERVACOES,RE_SGT,TROCA_PERMUTA,ESCALA_TOTVS,NOME,RE
2379,CLAUDETE MOURA,-,-,-,-,1434,986 286 454,-,-,-,,,CLAUDETE MOURA,1434
2380,-,-,-,-,-,-,-,-,-,-,,,CLAUDETE MOURA,1434
2381,Folgas:,Prev,-,12,Real,-,-,-,-,-,,,CLAUDETE MOURA,1434
2382,Dia,Escala,-,-,-,-,Código,-,Observações,Re,,,CLAUDETE MOURA,1434
2383,-,PREV,-,TOTVS,Jor,Entrada,Nº,Descrição,-,Resp,,,CLAUDETE MOURA,1434
2384,1,TRAB,D5,5-E,FOL,,C129,Troca Jorn,Por dia 17,2681,01768,01116,CLAUDETE MOURA,1434
2385,2,TRAB,D5,5-E,FOL,,C129,Troca Jorn,Por dia 18,2681,01768,01116,CLAUDETE MOURA,1434
2386,3,FOL,D5,5-E,FOL,,-,-,-,-,,01116,CLAUDETE MOURA,1434
2387,4,FOL,D5,5-E,FOL,,-,-,-,-,,01116,CLAUDETE MOURA,1434
2388,5,FOL,D5,5-E,08T,12:00:00,HEFT,HE folga TARDE,FALTA ROQUE,252,,01116,CLAUDETE MOURA,1434


In [87]:
# Primeira quinzena
RE(1434).iloc[:20,:]

,DIA,PREVISTO,ESCALA,TOTVS,JORNADA,ENTRADA,CODIGO,DESCRICAO,OBSERVACOES,RE_SGT,TROCA_PERMUTA,ESCALA_TOTVS,NOME,RE
2379,CLAUDETE MOURA,-,-,-,-,1434,986 286 454,-,-,-,,,CLAUDETE MOURA,1434
2380,-,-,-,-,-,-,-,-,-,-,,,CLAUDETE MOURA,1434
2381,Folgas:,Prev,-,12,Real,-,-,-,-,-,,,CLAUDETE MOURA,1434
2382,Dia,Escala,-,-,-,-,Código,-,Observações,Re,,,CLAUDETE MOURA,1434
2383,-,PREV,-,TOTVS,Jor,Entrada,Nº,Descrição,-,Resp,,,CLAUDETE MOURA,1434
2384,1,TRAB,D5,5-E,FOL,,C129,Troca Jorn,Por dia 17,2681,01768,01116,CLAUDETE MOURA,1434
2385,2,TRAB,D5,5-E,FOL,,C129,Troca Jorn,Por dia 18,2681,01768,01116,CLAUDETE MOURA,1434
2386,3,FOL,D5,5-E,FOL,,-,-,-,-,,01116,CLAUDETE MOURA,1434
2387,4,FOL,D5,5-E,FOL,,-,-,-,-,,01116,CLAUDETE MOURA,1434
2388,5,FOL,D5,5-E,08T,12:00:00,HEFT,HE folga TARDE,FALTA ROQUE,252,,01116,CLAUDETE MOURA,1434


In [88]:
# Criar a funcao de busca RMOP por nome
def OP(nome):
  return df[df['NOME'] == str(nome)]

In [89]:
  OP('ANA CAROLINA')

,DIA,PREVISTO,ESCALA,TOTVS,JORNADA,ENTRADA,CODIGO,DESCRICAO,OBSERVACOES,RE_SGT,TROCA_PERMUTA,ESCALA_TOTVS,NOME,RE
663,ANA CAROLINA,-,-,-,-,2922,982 917 027,-,-,-,,,ANA CAROLINA,2922
664,-,-,-,-,-,-,-,-,-,-,,,ANA CAROLINA,2922
665,Folgas:,Prev,-,10,Real,-,-,-,-,-,,,ANA CAROLINA,2922
666,Dia,Escala,-,-,-,-,Código,-,Observações,Re,,,ANA CAROLINA,2922
667,-,PREV,-,TOTVS,Jor,Entrada,Nº,Descrição,-,Resp,,,ANA CAROLINA,2922
668,1,FOL,Y04,_,FOL,,C208,Atestado,-,2681,,01601,ANA CAROLINA,2922
669,2,FOL,Y04,_,FOL,,C208,Atestado,-,2681,,01601,ANA CAROLINA,2922
670,3,TRAB,Y04,_,FAT,COLOCAR,C208,Atestado,-,2681,,01601,ANA CAROLINA,2922
671,4,TRAB,Y04,_,FAT,COLOCAR,C208,Atestado,-,2681,,01601,ANA CAROLINA,2922
672,5,TRAB,Y04,_,FAT,COLOCAR,C208,Atestado,-,2681,,01601,ANA CAROLINA,2922
